In [ ]:
"""
Web scrape for more info

1. speaker (birth year, place)
2. school

"""

In [14]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

import pandas as pd
import numpy as np
import pickle

import sys
sys.path.append('/Users/katiehuang/Documents/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

## 1. Load data

In [328]:
speech_df = pd.read_pickle('../dump/speech_df')
speech_df

,speaker,year,school,transcript,length
0,SIDDHARTHA MUKHERJEE,2018,Univ. of Southern California,I wish someone had told me at my own commenc...,14487
1,JESMYN WARD,2018,Tulane University,Persist. Be patient. Be well. Good morning....,14063
2,DAVID SEDARIS,2018,OBERLIN COLLEGE,"Thank you so much for having me, and for prese...",10587
3,NIGHT SHYAMALAN,2018,DREXEL UNIVERSITY,\nwhat's up dragons alright let's start\n\nwit...,26406
4,TERRY TEACHOUT,2018,HAMILTON HOLT SCHOOL,"I’m supposed to keep it short, and I approve o...",6288
...,...,...,...,...,...
436,WILLIAM ALLEN,1936,WHITE NORTHWESTERN UNIVERSITY,About all that a commencement orator can do fo...,14953
437,FRANKLIN D ROOSEVELT,1932,OGLETHORPE UNIVERSITY,"For me, as for you, this is a day of honorable...",16017
438,OPRAH WINFREY,1918,USC,Thank you Wallis Annenberg and a special thank...,15301
439,RALPH WALDO,1838,EMERSON HARVARD UNIVERSITY,"In this refulgent summer, it has been a luxury...",40403


## 2. Speaker info from Google
Scrape with Selenium

In [152]:
# Create list of speakers
speaker_list = list(speech_df.speaker)
len(speaker_list)

474

In [273]:
def google_speaker(speaker):
    
    keyword = speaker
    speaker_dict={}
    headers = ["speaker","profession","birth","born","age","origin"]

    # Selenium begins
    driver.get("https://www.google.com")
    search_bar = driver.find_element_by_xpath("//input[@name='q'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    #     print(speaker)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)

    # Set default value of result
    profession, birth, born, age, origin = np.nan, np.nan, np.nan, np.nan, np.nan

    # Find profession
    try:
        profession = driver.find_element_by_xpath("//div[@data-attrid='subtitle']").text
    except:
        pass

    if profession is np.nan:
        try:
            description = driver.find_element_by_xpath("//div[@data-attrid='description']").text
            profession = re.findall("(?<=was )(.*)(?= who)", description)[0]
        except:
            pass

    # Find birth info    
    try:
        birth = driver.find_element_by_xpath("//div[@class='rVusze']").text
    except:
        pass

    # Extract age and birth blace from birth info
    # Age
    try:
        age = re.findall("age \d+", birth)[0].split(' ')[-1]
    except:
        pass

    # Year born
    try:
        born = re.findall("Born: .* \d+", birth)[0].split(', ')[-1]
    except:
        pass

    # Birth place
    try:
        origin = " ".join(birth.split(', ')[-2:])
    except:
        pass

    speak_dict = dict(zip(headers,
                [speaker,profession,birth,born,age,origin]))

    return speak_dict

In [275]:
# Scrape
# results = []
# for speaker in speaker_list:
#     result = google_speaker(speaker)
#     results.append(result)

In [316]:
# Create DataFrame of speaker_info
speaker_info = pd.DataFrame(results)
# speaker_info

In [318]:
# Clean up the born column
speaker_info['born'] = speaker_info['born'].apply(str).apply(lambda x: x.split(' ')[0])
speaker_info.drop(columns=['birth'],inplace=True)
speaker_info

,speaker,profession,born,age,origin
0,SIDDHARTHA MUKHERJEE,American-Indian physician,1970,50,New Delhi India
1,JESMYN WARD,American novelist,1977,43,De Lisle MS
2,DAVID SEDARIS,American humorist,1956,64,Johnson City NY
3,TERRY TEACHOUT,American author,1956,65,Cape Girardeau MO
4,NIGHT SHYAMALAN,Filmmaker,1970,50,Mahé India
...,...,...,...,...,...
469,CARRIE CHAPMAN,an American women's suffrage leader,1859,NaN,Ripon WI
470,FRANKLIN D ROOSEVELT,32nd U.S. President,1882,NaN,Hyde Park NY
471,OPRAH WINFREY,American host,1954,67,Kosciusko MS
472,RALPH WALDO,American essayist,1803,NaN,Boston MA


In [326]:
speaker_info[speaker_info.duplicated(['speaker'])==True]

,speaker,profession,born,age,origin
31,SHERYL SANDBERG,Chief Operating Officer of Facebook,1969,51,Washington D.C.
55,CHIMAMANDA NGOZI ADICHIE,Writer,1977,43,Enugu Nigeria
69,SHERYL SANDBERG,Chief Operating Officer of Facebook,1969,51,Washington D.C.
79,MINDY KALING,American actress,1979,41,Cambridge MA
100,ED HELMS,American actor,1974,47,Atlanta GA
105,JOHN GREEN,American author,1977,43,Indianapolis IN
111,NEIL DEGRASSE,American astrophysicist,1958,62,New York NY
114,STEPHEN COLBERT,American comedian,1964,56,Washington D.C.
130,ED HELMS,American actor,1974,47,Atlanta GA
131,DAVID BROOKS,Commentator,1961,59,Toronto Canada


## 3. School info from Google
Location (latitude, longitude)

In [330]:
# Create list of speakers
school_list = list(speech_df.school)
len(school_list)

441

In [419]:
def google_school(school):
    
    school = school
    keyword = school + ' coordinate dd'
    school_dict={}
    headers = ["school","coord","latitude","longitude"]

    # Selenium begins
    driver.get("https://www.google.com")
    search_bar = driver.find_element_by_xpath("//input[@name='q'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(keyword)
    #     print(speaker)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(2)

    # Set default value of result
    coord,latitude,longitude = np.nan,np.nan,np.nan

    # Find coordinate decimal degrees
    try:
        coord = driver.find_element_by_xpath("//span[@class='hgKElc']").text
    except:
        pass
    
    
    # Extract latitude and longitude
    try:
        latitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[0]
        longitude = re.findall('[-+]?\d*\.?[-+]?\d+', coord)[1]
    except:
        pass

    school_dict = dict(zip(headers,
                [school,coord,latitude,longitude]))

    return school_dict

In [420]:
# # Scrape
# results_school = []
# for school in school_list:
#     result = google_school(school)
#     results_school.append(result)

In [434]:
# Create DataFrame of speaker_info
school_info = pd.DataFrame(results_school)
school_info.drop(columns=['coord'],inplace=True)
school_info

,school,latitude,longitude
0,Univ. of Southern California,34.0205,-118.2856
1,Tulane University,29.9353,-90.1227
2,OBERLIN COLLEGE,41.2929,-82.2186
3,DREXEL UNIVERSITY,39.9564,-75.1887
4,HAMILTON HOLT SCHOOL,NaN,NaN
...,...,...,...
436,WHITE NORTHWESTERN UNIVERSITY,NaN,NaN
437,OGLETHORPE UNIVERSITY,NaN,NaN
438,USC,34.0205,-118.2856
439,EMERSON HARVARD UNIVERSITY,NaN,NaN


In [433]:
# Examine missing values
school_info[school_info.coord.isna()==True]

,school,coord,latitude,longitude
4,HAMILTON HOLT SCHOOL,NaN,NaN,NaN
6,Darmouth,NaN,NaN,NaN
18,Agnes Scott College,NaN,NaN,NaN
25,SCAD,NaN,NaN,NaN
27,Three Rivers College,NaN,NaN,NaN
...,...,...,...,...
431,EUREKA COLLEGE,NaN,NaN,NaN
435,CATT SWEET BRIAR COLLEGE,NaN,NaN,NaN
436,WHITE NORTHWESTERN UNIVERSITY,NaN,NaN,NaN
437,OGLETHORPE UNIVERSITY,NaN,NaN,NaN


## Testing section